# Importing libraries

In [1]:
import os
import sys
import glob
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession,functions, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.pandas as ps


# Starting Spark Session and Creating a Spark Session Called Pizza

In [3]:
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("PizzaRestaurant") \
        .config("spark.some.config.option", "config-value") \
        .enableHiveSupport() \
        .getOrCreate()

In [4]:
spark

# Browsing .csv PizzaRestaurant files

In [5]:
path = r"D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales"


if os.path.exists(path):
    print(f'The file {path} exists')
else: 
    print(f"The file {path} does not exist ")

filenames = glob.glob(path + '\*.csv')

for file in filenames:
    print("\nFile names:", file)


The file D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales exists

File names: D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\orders.csv

File names: D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\order_details.csv

File names: D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\pizzas.csv

File names: D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\pizza_types.csv


# Reading csv files into Spark DataFrames

In [6]:
PathOrders = 'D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\orders.csv'
PathOrderDetails = 'D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\order_details.csv'
PathPizzas = 'D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\pizzas.csv'
PathPizzaTypes = 'D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\pizza_types.csv'

dfPizzaOrders = spark.read.csv(PathOrders, header=True, inferSchema=True)
dfPizzaOrderDetails = spark.read.csv(PathOrderDetails, header=True, inferSchema=True)
dfPizzas = spark.read.csv(PathPizzas, header=True, inferSchema=True)
dfPizzaTypes = spark.read.csv(PathPizzaTypes, header=True, inferSchema=True)


# Data Exploration

In [7]:
dfPizzaOrders.show(truncate=False)

+--------+----------+-------------------+
|order_id|date      |time               |
+--------+----------+-------------------+
|1       |2015-01-01|2023-06-12 11:38:36|
|2       |2015-01-01|2023-06-12 11:57:40|
|3       |2015-01-01|2023-06-12 12:12:28|
|4       |2015-01-01|2023-06-12 12:16:31|
|5       |2015-01-01|2023-06-12 12:21:30|
|6       |2015-01-01|2023-06-12 12:29:36|
|7       |2015-01-01|2023-06-12 12:50:37|
|8       |2015-01-01|2023-06-12 12:51:37|
|9       |2015-01-01|2023-06-12 12:52:01|
|10      |2015-01-01|2023-06-12 13:00:15|
|11      |2015-01-01|2023-06-12 13:02:59|
|12      |2015-01-01|2023-06-12 13:04:41|
|13      |2015-01-01|2023-06-12 13:11:55|
|14      |2015-01-01|2023-06-12 13:14:19|
|15      |2015-01-01|2023-06-12 13:33:00|
|16      |2015-01-01|2023-06-12 13:34:07|
|17      |2015-01-01|2023-06-12 13:53:00|
|18      |2015-01-01|2023-06-12 13:57:08|
|19      |2015-01-01|2023-06-12 13:59:09|
|20      |2015-01-01|2023-06-12 14:03:08|
+--------+----------+-------------

In [8]:
dfPizzaOrderDetails.show(truncate=False)

+----------------+--------+--------------+--------+
|order_details_id|order_id|pizza_id      |quantity|
+----------------+--------+--------------+--------+
|1               |1       |hawaiian_m    |1       |
|2               |2       |classic_dlx_m |1       |
|3               |2       |five_cheese_l |1       |
|4               |2       |ital_supr_l   |1       |
|5               |2       |mexicana_m    |1       |
|6               |2       |thai_ckn_l    |1       |
|7               |3       |ital_supr_m   |1       |
|8               |3       |prsc_argla_l  |1       |
|9               |4       |ital_supr_m   |1       |
|10              |5       |ital_supr_m   |1       |
|11              |6       |bbq_ckn_s     |1       |
|12              |6       |the_greek_s   |1       |
|13              |7       |spinach_supr_s|1       |
|14              |8       |spinach_supr_s|1       |
|15              |9       |classic_dlx_s |1       |
|16              |9       |green_garden_s|1       |
|17         

In [9]:
dfPizzas.show(truncate=False)

+-------------+-------------+----+-----+
|pizza_id     |pizza_type_id|size|price|
+-------------+-------------+----+-----+
|bbq_ckn_s    |bbq_ckn      |S   |12.75|
|bbq_ckn_m    |bbq_ckn      |M   |16.75|
|bbq_ckn_l    |bbq_ckn      |L   |20.75|
|cali_ckn_s   |cali_ckn     |S   |12.75|
|cali_ckn_m   |cali_ckn     |M   |16.75|
|cali_ckn_l   |cali_ckn     |L   |20.75|
|ckn_alfredo_s|ckn_alfredo  |S   |12.75|
|ckn_alfredo_m|ckn_alfredo  |M   |16.75|
|ckn_alfredo_l|ckn_alfredo  |L   |20.75|
|ckn_pesto_s  |ckn_pesto    |S   |12.75|
|ckn_pesto_m  |ckn_pesto    |M   |16.75|
|ckn_pesto_l  |ckn_pesto    |L   |20.75|
|southw_ckn_s |southw_ckn   |S   |12.75|
|southw_ckn_m |southw_ckn   |M   |16.75|
|southw_ckn_l |southw_ckn   |L   |20.75|
|thai_ckn_s   |thai_ckn     |S   |12.75|
|thai_ckn_m   |thai_ckn     |M   |16.75|
|thai_ckn_l   |thai_ckn     |L   |20.75|
|big_meat_s   |big_meat     |S   |12.0 |
|big_meat_m   |big_meat     |M   |16.0 |
+-------------+-------------+----+-----+
only showing top

In [10]:
dfPizzaTypes.show(truncate=False)

+-------------+------------------------------------------+--------+--------------------------------------------------------------------------------------------+
|pizza_type_id|name                                      |category|ingredients                                                                                 |
+-------------+------------------------------------------+--------+--------------------------------------------------------------------------------------------+
|bbq_ckn      |The Barbecue Chicken Pizza                |Chicken |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce         |
|cali_ckn     |The California Chicken Pizza              |Chicken |Chicken, Artichoke, Spinach, Garlic, Jalapeno Peppers, Fontina Cheese, Gouda Cheese         |
|ckn_alfredo  |The Chicken Alfredo Pizza                 |Chicken |Chicken, Red Onions, Red Peppers, Mushrooms, Asiago Cheese, Alfredo Sauce                   |
|ckn_pesto    |The Chicken Pesto P

# Combining DataFrames

In [11]:
dfPizzaOrdersAll = dfPizzaOrders.join(dfPizzaOrderDetails,dfPizzaOrders.order_id == dfPizzaOrderDetails.order_id, "inner" )
dfPizzaOrdersAll.show(truncate=False)

+--------+----------+-------------------+----------------+--------+--------------+--------+
|order_id|date      |time               |order_details_id|order_id|pizza_id      |quantity|
+--------+----------+-------------------+----------------+--------+--------------+--------+
|1       |2015-01-01|2023-06-12 11:38:36|1               |1       |hawaiian_m    |1       |
|2       |2015-01-01|2023-06-12 11:57:40|2               |2       |classic_dlx_m |1       |
|2       |2015-01-01|2023-06-12 11:57:40|3               |2       |five_cheese_l |1       |
|2       |2015-01-01|2023-06-12 11:57:40|4               |2       |ital_supr_l   |1       |
|2       |2015-01-01|2023-06-12 11:57:40|5               |2       |mexicana_m    |1       |
|2       |2015-01-01|2023-06-12 11:57:40|6               |2       |thai_ckn_l    |1       |
|3       |2015-01-01|2023-06-12 12:12:28|7               |3       |ital_supr_m   |1       |
|3       |2015-01-01|2023-06-12 12:12:28|8               |3       |prsc_argla_l 

In [12]:
dfPizzaOrdersAll = dfPizzaOrdersAll.select("pizza_id","quantity","date","time")
dfPizzaOrdersAll.show()

+--------------+--------+----------+-------------------+
|      pizza_id|quantity|      date|               time|
+--------------+--------+----------+-------------------+
|    hawaiian_m|       1|2015-01-01|2023-06-12 11:38:36|
| classic_dlx_m|       1|2015-01-01|2023-06-12 11:57:40|
| five_cheese_l|       1|2015-01-01|2023-06-12 11:57:40|
|   ital_supr_l|       1|2015-01-01|2023-06-12 11:57:40|
|    mexicana_m|       1|2015-01-01|2023-06-12 11:57:40|
|    thai_ckn_l|       1|2015-01-01|2023-06-12 11:57:40|
|   ital_supr_m|       1|2015-01-01|2023-06-12 12:12:28|
|  prsc_argla_l|       1|2015-01-01|2023-06-12 12:12:28|
|   ital_supr_m|       1|2015-01-01|2023-06-12 12:16:31|
|   ital_supr_m|       1|2015-01-01|2023-06-12 12:21:30|
|     bbq_ckn_s|       1|2015-01-01|2023-06-12 12:29:36|
|   the_greek_s|       1|2015-01-01|2023-06-12 12:29:36|
|spinach_supr_s|       1|2015-01-01|2023-06-12 12:50:37|
|spinach_supr_s|       1|2015-01-01|2023-06-12 12:51:37|
| classic_dlx_s|       1|2015-0

In [13]:
dfPizzaAll = dfPizzas.join(dfPizzaTypes,dfPizzas.pizza_type_id == dfPizzaTypes.pizza_type_id, "inner" )
dfPizzaAll = dfPizzaAll.select("pizza_id","size","price","name","category","ingredients")
dfPizzaAll.show()


+-------------+----+-----+--------------------+--------+--------------------+
|     pizza_id|size|price|                name|category|         ingredients|
+-------------+----+-----+--------------------+--------+--------------------+
|    bbq_ckn_l|   L|20.75|The Barbecue Chic...| Chicken|Barbecued Chicken...|
|    bbq_ckn_m|   M|16.75|The Barbecue Chic...| Chicken|Barbecued Chicken...|
|    bbq_ckn_s|   S|12.75|The Barbecue Chic...| Chicken|Barbecued Chicken...|
|   cali_ckn_l|   L|20.75|The California Ch...| Chicken|Chicken, Artichok...|
|   cali_ckn_m|   M|16.75|The California Ch...| Chicken|Chicken, Artichok...|
|   cali_ckn_s|   S|12.75|The California Ch...| Chicken|Chicken, Artichok...|
|ckn_alfredo_l|   L|20.75|The Chicken Alfre...| Chicken|Chicken, Red Onio...|
|ckn_alfredo_m|   M|16.75|The Chicken Alfre...| Chicken|Chicken, Red Onio...|
|ckn_alfredo_s|   S|12.75|The Chicken Alfre...| Chicken|Chicken, Red Onio...|
|  ckn_pesto_l|   L|20.75|The Chicken Pesto...| Chicken|Chicken,

In [14]:
dfPizza = dfPizzaAll.join(dfPizzaOrdersAll,dfPizzaAll.pizza_id == dfPizzaOrdersAll.pizza_id, "left" )
dfPizza = dfPizza.select(col("name").alias("Name"), 
                col("category").alias("Category"),
                col("ingredients").alias("Ingredients"),
                col("size").alias("Size"),
                col("price").alias("Price"),
                col("quantity").alias("Quantity"),
                col("date").alias("Date"),
                col("time").alias("Time"))

dfPizza.show(truncate=False)


+--------------------------+--------+-----------------------------------------------------------------------------------+----+-----+--------+----------+-------------------+
|Name                      |Category|Ingredients                                                                        |Size|Price|Quantity|Date      |Time               |
+--------------------------+--------+-----------------------------------------------------------------------------------+----+-----+--------+----------+-------------------+
|The Barbecue Chicken Pizza|Chicken |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|1       |2015-01-01|2023-06-12 13:02:59|
|The Barbecue Chicken Pizza|Chicken |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|1       |2015-01-01|2023-06-12 13:53:00|
|The Barbecue Chicken Pizza|Chicken |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|1  

# Data Cleaning

### Looking for null values

In [15]:
dfPizzaNull = dfPizza.select("Name", "Category", "Ingredients", "Size", "Price", "Quantity")

dfPizzaNull.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dfPizzaNull.columns]
   ).show()

+----+--------+-----------+----+-----+--------+
|Name|Category|Ingredients|Size|Price|Quantity|
+----+--------+-----------+----+-----+--------+
|   0|       0|          0|   0|    0|       5|
+----+--------+-----------+----+-----+--------+



##### There are a few Null values in Quantity, but it means that there were no orders for these pizza

In [16]:
dfPizza.filter(col("Quantity").isNull()).show()

+--------------------+--------+--------------------+----+-----+--------+----+----+
|                Name|Category|         Ingredients|Size|Price|Quantity|Date|Time|
+--------------------+--------+--------------------+----+-----+--------+----+----+
|The Four Cheese P...|  Veggie|Ricotta Cheese, G...|   S|11.75|    null|null|null|
|  The Big Meat Pizza| Classic|Bacon, Pepperoni,...|   L| 20.5|    null|null|null|
|The Five Cheese P...|  Veggie|Mozzarella Cheese...|   S| 12.5|    null|null|null|
|  The Big Meat Pizza| Classic|Bacon, Pepperoni,...|   M| 16.0|    null|null|null|
|The Five Cheese P...|  Veggie|Mozzarella Cheese...|   M| 15.5|    null|null|null|
+--------------------+--------+--------------------+----+-----+--------+----+----+



### Dealing with duplicate values

In [17]:
print("Counting rows {}".format(dfPizza.count()))

Counting rows 48625


In [18]:
dfPizzaDuplicates = dfPizza.groupBy(dfPizza.columns).count().filter(col("count") > 1)
dfPizzaDuplicates.show()

+----+--------+-----------+----+-----+--------+----+----+-----+
|Name|Category|Ingredients|Size|Price|Quantity|Date|Time|count|
+----+--------+-----------+----+-----+--------+----+----+-----+
+----+--------+-----------+----+-----+--------+----+----+-----+



### Browsing for distinct values in each column

In [19]:
print("Counting distinct rows {}".format(dfPizza.distinct().count()))

Counting distinct rows 48625


In [20]:
dfPizza.agg(*(countDistinct(col(c)).alias(c) for c in dfPizza.columns)).show()

+----+--------+-----------+----+-----+--------+----+-----+
|Name|Category|Ingredients|Size|Price|Quantity|Date| Time|
+----+--------+-----------+----+-----+--------+----+-----+
|  32|       4|         32|   5|   27|       4| 358|16382|
+----+--------+-----------+----+-----+--------+----+-----+



### Columns

In [21]:
print("There are {} columns in the DataFrame and their names are {}".format(len(dfPizza.columns),
                                                                              dfPizza.columns))

There are 8 columns in the DataFrame and their names are ['Name', 'Category', 'Ingredients', 'Size', 'Price', 'Quantity', 'Date', 'Time']


### Datatypes

In [22]:
dfPizza.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Ingredients: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)



In [23]:
dfPizza = dfPizza.withColumn("Price",col("Price").cast(DecimalType(18,2)))

### Summary

In [24]:
dfPizza.describe("Price").show()

+-------+-----------------+
|summary|            Price|
+-------+-----------------+
|  count|            48625|
|   mean|        16.494004|
| stddev|3.621761197368334|
|    min|             9.75|
|    max|            35.95|
+-------+-----------------+



In [25]:
dfPizza.dtypes

[('Name', 'string'),
 ('Category', 'string'),
 ('Ingredients', 'string'),
 ('Size', 'string'),
 ('Price', 'decimal(18,2)'),
 ('Quantity', 'int'),
 ('Date', 'date'),
 ('Time', 'timestamp')]

In [26]:
print("Shape of DataFrame: {}, {} ".format(dfPizza.count(),
                                          len(dfPizza.columns)))

Shape of DataFrame: 48625, 8 


# Data Analysis

### Creating Total Amount

In [27]:
def CreateMultpiplication(x, y):
    return x * y

dfPizza = dfPizza.withColumn("Total Amount",CreateMultpiplication(col("Quantity"),col("Price")) )
dfPizza = dfPizza.select('Name',
 'Category',
 'Ingredients',
 'Size',
 'Price',
 'Quantity',
 'Total Amount',
 'Date',
 'Time')

dfPizza.show(truncate=False)

+-------------------------+--------+-----------------------------------------------------------------------+----+-----+--------+------------+----------+-------------------+
|Name                     |Category|Ingredients                                                            |Size|Price|Quantity|Total Amount|Date      |Time               |
+-------------------------+--------+-----------------------------------------------------------------------+----+-----+--------+------------+----------+-------------------+
|The Italian Supreme Pizza|Supreme |Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic|S   |12.50|1       |12.50       |2015-12-30|2023-06-12 12:45:20|
|The Italian Supreme Pizza|Supreme |Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic|S   |12.50|1       |12.50       |2015-12-27|2023-06-12 16:03:36|
|The Italian Supreme Pizza|Supreme |Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic|S   |12.50|1       |12.50  

### Creating upper case of Category

In [28]:
def upperCase(str):
    return str.upper()

upperCaseUDF = udf(lambda z:upperCase(z), StringType())

dfPizza = dfPizza.withColumn("Category",upperCaseUDF(col("Category")))
dfPizza.show(truncate=False)

+--------------------------+--------+-----------------------------------------------------------------------------------+----+-----+--------+------------+----------+-------------------+
|Name                      |Category|Ingredients                                                                        |Size|Price|Quantity|Total Amount|Date      |Time               |
+--------------------------+--------+-----------------------------------------------------------------------------------+----+-----+--------+------------+----------+-------------------+
|The Barbecue Chicken Pizza|CHICKEN |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|1       |20.75       |2015-01-01|2023-06-12 13:02:59|
|The Barbecue Chicken Pizza|CHICKEN |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|1       |20.75       |2015-01-01|2023-06-12 13:53:00|
|The Barbecue Chicken Pizza|CHICKEN |Barbecued Chicken, Red Peppers, G

### Summing of Total Amount by Category and Pizza Name

In [29]:
dfPizzaCategoryByTotalAmount = dfPizza.groupBy("Category").sum("Total Amount").orderBy(desc("sum(Total Amount)"))
dfPizzaCategoryByTotalAmount.select("Category",col("sum(Total Amount)").alias("Total Amount")).show(truncate=False)

+--------+------------+
|Category|Total Amount|
+--------+------------+
|CLASSIC |220053.10   |
|SUPREME |208197.00   |
|CHICKEN |195919.50   |
|VEGGIE  |193690.45   |
+--------+------------+



In [30]:
dfPizzaPizzaNameByTotalAmount = dfPizza.groupBy("Name").sum("Total Amount")
windowSpec = Window.orderBy(desc("sum(Total Amount)"))
dfPizzaPizzaNameByTotalAmount.withColumn("Ranking", row_number().over(windowSpec)).show(truncate=False)

+---------------------------------+-----------------+-------+
|Name                             |sum(Total Amount)|Ranking|
+---------------------------------+-----------------+-------+
|The Thai Chicken Pizza           |43434.25         |1      |
|The Barbecue Chicken Pizza       |42768.00         |2      |
|The California Chicken Pizza     |41409.50         |3      |
|The Classic Deluxe Pizza         |38180.50         |4      |
|The Spicy Italian Pizza          |34831.25         |5      |
|The Southwest Chicken Pizza      |34705.75         |6      |
|The Italian Supreme Pizza        |33476.75         |7      |
|The Hawaiian Pizza               |32273.25         |8      |
|The Four Cheese Pizza            |32265.70         |9      |
|The Sicilian Pizza               |30940.50         |10     |
|The Pepperoni Pizza              |30161.75         |11     |
|The Greek Pizza                  |28454.10         |12     |
|The Mexicana Pizza               |26780.75         |13     |
|The Fiv

### Grouping of Pizza Name 

In [31]:
dfPizzaPizzaNameByTotalAmount.withColumn("Grouping",ntile(5).over(windowSpec)).show(truncate=False)

dfPizzaPizzaNameByTotalAmountGrouping =  dfPizzaPizzaNameByTotalAmount.withColumn("Grouping",ntile(5).over(windowSpec))


+---------------------------------+-----------------+--------+
|Name                             |sum(Total Amount)|Grouping|
+---------------------------------+-----------------+--------+
|The Thai Chicken Pizza           |43434.25         |1       |
|The Barbecue Chicken Pizza       |42768.00         |1       |
|The California Chicken Pizza     |41409.50         |1       |
|The Classic Deluxe Pizza         |38180.50         |1       |
|The Spicy Italian Pizza          |34831.25         |1       |
|The Southwest Chicken Pizza      |34705.75         |1       |
|The Italian Supreme Pizza        |33476.75         |1       |
|The Hawaiian Pizza               |32273.25         |2       |
|The Four Cheese Pizza            |32265.70         |2       |
|The Sicilian Pizza               |30940.50         |2       |
|The Pepperoni Pizza              |30161.75         |2       |
|The Greek Pizza                  |28454.10         |2       |
|The Mexicana Pizza               |26780.75         |2 

In [32]:
dfPizzaPizzaNameByTotalAmountGrouping.filter(dfPizzaPizzaNameByTotalAmountGrouping["Grouping"] == 1).show(truncate=False)

+----------------------------+-----------------+--------+
|Name                        |sum(Total Amount)|Grouping|
+----------------------------+-----------------+--------+
|The Thai Chicken Pizza      |43434.25         |1       |
|The Barbecue Chicken Pizza  |42768.00         |1       |
|The California Chicken Pizza|41409.50         |1       |
|The Classic Deluxe Pizza    |38180.50         |1       |
|The Spicy Italian Pizza     |34831.25         |1       |
|The Southwest Chicken Pizza |34705.75         |1       |
|The Italian Supreme Pizza   |33476.75         |1       |
+----------------------------+-----------------+--------+



In [33]:
dfPizzaPizzaNameByTotalAmountGrouping.filter(dfPizzaPizzaNameByTotalAmountGrouping["Grouping"] == 5).show(truncate=False)

+-------------------------+-----------------+--------+
|Name                     |sum(Total Amount)|Grouping|
+-------------------------+-----------------+--------+
|The Calabrese Pizza      |15934.25         |5       |
|The Spinach Pesto Pizza  |15596.00         |5       |
|The Mediterranean Pizza  |15360.50         |5       |
|The Spinach Supreme Pizza|15277.75         |5       |
|The Green Garden Pizza   |13955.75         |5       |
|The Brie Carre Pizza     |11588.50         |5       |
+-------------------------+-----------------+--------+



### Running Total by Month

In [34]:
dfPizzaGrouping = dfPizza.select(year(col("Date")),
               month(col("Date")),
               col("Total Amount"))


dfPizzaGrouping = dfPizzaGrouping.groupBy("year(Date)","month(Date)").sum("Total Amount")
dfPizzaGrouping = dfPizzaGrouping.na.drop()
dfPizzaGrouping = dfPizzaGrouping.select(col("year(Date)").alias("Year"),
                       col("month(Date)").alias("Month"),
                       col("sum(Total Amount)").alias("Total Amount"))

dfPizzaGrouping.orderBy("Month").show(truncate=False)


+----+-----+------------+
|Year|Month|Total Amount|
+----+-----+------------+
|2015|1    |69793.30    |
|2015|2    |65159.60    |
|2015|3    |70397.10    |
|2015|4    |68736.80    |
|2015|5    |71402.75    |
|2015|6    |68230.20    |
|2015|7    |72557.90    |
|2015|8    |68278.25    |
|2015|9    |64180.05    |
|2015|10   |64027.60    |
|2015|11   |70395.35    |
|2015|12   |64701.15    |
+----+-----+------------+



In [35]:
windowSpec = Window.partitionBy().orderBy("Month")
windowval = (Window.partitionBy().orderBy("Month").rangeBetween(Window.unboundedPreceding,0))

dfPizzaGrouping = dfPizzaGrouping.select("Year",
                                         "Month",
                                         concat_ws("-",col("Month"),col("Year")).alias("Month-Year"),
                                         "Total Amount")
dfPizzaGrouping = dfPizzaGrouping.withColumn("Lag", lag(col("Total Amount"), offset=1, default=0).over(windowSpec))
dfPizzaGrouping = dfPizzaGrouping.withColumn("Diff",
                                             when(isnull(col("Total Amount") - col("Lag")),0)
                                                  .otherwise(col("Total Amount") - col("Lag")))
dfPizzaGrouping = dfPizzaGrouping.withColumn("Lead", lead(col("Total Amount"), offset=1,default = 0).over(windowSpec))
dfPizzaGrouping = dfPizzaGrouping.withColumn("Cummluative Sum", sum(col("Total Amount")).over(windowval))

dfPizzaGrouping.select("Month-Year",
                       "Total Amount",
                       "Lag",
                       "Diff",
                       "Lead",
                       "Cummluative Sum").show(truncate=False )




+----------+------------+--------+--------+--------+---------------+
|Month-Year|Total Amount|Lag     |Diff    |Lead    |Cummluative Sum|
+----------+------------+--------+--------+--------+---------------+
|1-2015    |69793.30    |0.00    |69793.30|65159.60|69793.30       |
|2-2015    |65159.60    |69793.30|-4633.70|70397.10|134952.90      |
|3-2015    |70397.10    |65159.60|5237.50 |68736.80|205350.00      |
|4-2015    |68736.80    |70397.10|-1660.30|71402.75|274086.80      |
|5-2015    |71402.75    |68736.80|2665.95 |68230.20|345489.55      |
|6-2015    |68230.20    |71402.75|-3172.55|72557.90|413719.75      |
|7-2015    |72557.90    |68230.20|4327.70 |68278.25|486277.65      |
|8-2015    |68278.25    |72557.90|-4279.65|64180.05|554555.90      |
|9-2015    |64180.05    |68278.25|-4098.20|64027.60|618735.95      |
|10-2015   |64027.60    |64180.05|-152.45 |70395.35|682763.55      |
|11-2015   |70395.35    |64027.60|6367.75 |64701.15|753158.90      |
|12-2015   |64701.15    |70395.35|

### The quantity of orders in terms of Pizza and its size

In [36]:
dfOrderedPizza = dfPizza.na.drop()

dfOrderedPizza.groupBy("Name").count().orderBy(desc("count")).show(40,100)

+------------------------------------------+-----+
|                                      Name|count|
+------------------------------------------+-----+
|                  The Classic Deluxe Pizza| 2416|
|                The Barbecue Chicken Pizza| 2372|
|                        The Hawaiian Pizza| 2370|
|                       The Pepperoni Pizza| 2369|
|                    The Thai Chicken Pizza| 2315|
|              The California Chicken Pizza| 2302|
|                   The Spicy Italian Pizza| 1887|
|                        The Sicilian Pizza| 1887|
|               The Southwest Chicken Pizza| 1885|
|                     The Four Cheese Pizza| 1850|
|                 The Italian Supreme Pizza| 1849|
|                        The Big Meat Pizza| 1811|
|         The Vegetables + Vegetables Pizza| 1510|
|                        The Mexicana Pizza| 1456|
|                      The Napolitana Pizza| 1451|
|                The Spinach and Feta Pizza| 1432|
|          The Prosciutto and A

In [37]:
dfOrderedPizza.groupBy("Size").count().orderBy(desc("count")).show(truncate=False)

+----+-----+
|Size|count|
+----+-----+
|L   |18526|
|M   |15385|
|S   |14137|
|XL  |544  |
|XXL |28   |
+----+-----+



### Quantity of orders in a day and which pizza was ordered most of the time

In [38]:
dfPizzaWithoutTime  = dfPizza.select('Name',
 'Category',
 'Ingredients',
 'Size',
 'Price',
 'Quantity',
 'Total Amount',
 'Date')

dfPizzaDayMonthYear = dfPizzaWithoutTime.withColumn("Day", dayofmonth(col("Date"))) \
        .withColumn("Month", month(col("Date"))) \
        .withColumn("Year", year(col("Date")))

dfPizzaDayMonthYear = dfPizzaDayMonthYear.na.drop()

dfPizzaDayMonthYear.groupBy("Day").count().orderBy("Day").show(truncate=False)

+---+-----+
|Day|count|
+---+-----+
|1  |1782 |
|2  |1624 |
|3  |1660 |
|4  |1773 |
|5  |1427 |
|6  |1673 |
|7  |1647 |
|8  |1646 |
|9  |1656 |
|10 |1601 |
|11 |1655 |
|12 |1443 |
|13 |1643 |
|14 |1643 |
|15 |1844 |
|16 |1609 |
|17 |1649 |
|18 |1589 |
|19 |1463 |
|20 |1650 |
|21 |1560 |
|22 |1484 |
|23 |1595 |
|24 |1546 |
|25 |1245 |
|26 |1504 |
|27 |1763 |
|28 |1450 |
|29 |1361 |
|30 |1439 |
|31 |996  |
+---+-----+



In [39]:
dfPizzaDayMonthYear.groupBy("Day","Name").count().orderBy("Day").show(truncate=False)

+---+------------------------------------------+-----+
|Day|Name                                      |count|
+---+------------------------------------------+-----+
|1  |The Italian Capocollo Pizza               |59   |
|1  |The Brie Carre Pizza                      |11   |
|1  |The Classic Deluxe Pizza                  |94   |
|1  |The Big Meat Pizza                        |55   |
|1  |The Mexicana Pizza                        |45   |
|1  |The Italian Vegetables Pizza              |32   |
|1  |The Spinach and Feta Pizza                |51   |
|1  |The Spicy Italian Pizza                   |58   |
|1  |The Chicken Pesto Pizza                   |32   |
|1  |The California Chicken Pizza              |75   |
|1  |The Mediterranean Pizza                   |31   |
|1  |The Pepperoni Pizza                       |86   |
|1  |The Southwest Chicken Pizza               |78   |
|1  |The Hawaiian Pizza                        |81   |
|1  |The Greek Pizza                           |47   |
|1  |The C

### Total Amount of Pizza in each day

In [40]:
dfPizzaDayMonthYear.groupBy("Day").sum("Total Amount").orderBy("Day").show(truncate=False)
# put alias

+---+-----------------+
|Day|sum(Total Amount)|
+---+-----------------+
|1  |29608.55         |
|2  |27495.40         |
|3  |27993.15         |
|4  |29675.65         |
|5  |23988.40         |
|6  |28097.45         |
|7  |27636.40         |
|8  |27976.90         |
|9  |27963.90         |
|10 |26967.95         |
|11 |27756.20         |
|12 |24169.85         |
|13 |27665.10         |
|14 |27727.25         |
|15 |30588.90         |
|16 |26890.35         |
|17 |27969.25         |
|18 |26913.70         |
|19 |24755.70         |
|20 |27811.20         |
|21 |26028.70         |
|22 |25004.80         |
|23 |27137.05         |
|24 |26098.60         |
|25 |20875.80         |
|26 |25247.35         |
|27 |29655.95         |
|28 |24154.85         |
|29 |23045.40         |
|30 |24230.60         |
|31 |16729.70         |
+---+-----------------+



In [41]:
dfPizzaDayMonthYear.groupBy("Day","Name").sum("Total Amount").orderBy("Day").show(truncate=False)
# put alias

+---+------------------------------------------+-----------------+
|Day|Name                                      |sum(Total Amount)|
+---+------------------------------------------+-----------------+
|1  |The Italian Capocollo Pizza               |1022.50          |
|1  |The Brie Carre Pizza                      |260.15           |
|1  |The Classic Deluxe Pizza                  |1490.00          |
|1  |The Big Meat Pizza                        |708.00           |
|1  |The Mexicana Pizza                        |876.25           |
|1  |The Italian Vegetables Pizza              |529.50           |
|1  |The Spinach and Feta Pizza                |840.00           |
|1  |The Spicy Italian Pizza                   |1049.50          |
|1  |The Chicken Pesto Pizza                   |576.75           |
|1  |The California Chicken Pizza              |1320.25          |
|1  |The Mediterranean Pizza                   |535.25           |
|1  |The Pepperoni Pizza                       |1105.75       

### Looking for the greatest value of these and minimum and which day

### Counting orders in each month in the name of month

#### Ideas in the future:

In [ ]:
#from pyspark.sql import functions as F


#df2 = dfPizza.select("Ingredients", split("Ingredients", ", ").alias('col2'))

#df2.show(truncate=False)
#df_sizes = df2.select(F.size('col2').alias('col2'))
#df_max = df_sizes.agg(F.max('col2'))
#nb_columns = df_max.collect()[0][0]

#df_result = df2.select('Ingredients', *[df2['col2'][i] for i in range(nb_columns)])
#df_result.show(truncate=False)

In [ ]:
#If you don't know the number of columns:
#df_sizes = df2.select(F.size('col2').alias('col2'))
#df_max = df_sizes.agg(F.max('col2'))
#nb_columns = df_max.collect()[0][0]

#df_result = df2.select('col1', *[df2['col2'][i] for i in range(nb_columns)])
#df_result.show()
#from pyspark.sql.functions import *

In [42]:
#Things to be considered in next project
# dealing with these https://www.datasciencemadesimple.com/get-month-year-and-quarter-from-date-in-pyspark/, 
# https://www.datasciencemadesimple.com/get-day-of-month-day-of-year-day-of-week-from-date-in-pyspark/
#consdering day of week? https://www.datasciencemadesimple.com/get-day-of-month-day-of-year-day-of-week-from-date-in-pyspark/
# browsing all materials that I have so as to select!!
## thinking what should be done with pizza without orders
# thinking about one day - sales amount in one day - the orders of quantity in one day - workig on it
## in which month the quantity of orders of pizza are the greatest and which pizza are ordered most!
## how many pizzas were ordered based on size - prices of each pizza , total amount by pizza 
## using UDF (put  example and remove digit at the beginning) & analytical functions
#Looking for values with only one ingredient for example salami using wildcard
# using pandas and look for spicy pizza or ingredients
#case when with size and is null where it is not available
#https://sparkbyexamples.com/pyspark/pyspark-groupby-explained-with-example/
# https://sparkbyexamples.com/pyspark/pyspark-window-functions/
# https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/ 
# Considering next transformations to be used
# max and min of each borough 
## using pandas / pivot
# using inspirations to run this project from .doc files 
# https://pub.towardsai.net/exploratory-data-analysis-eda-using-pyspark-b43e71fcec9f
#https://dev.to/kinyungu_denis/apache-pyspark-for-data-engineering-3phi